In [16]:
#bgr8 to jpeg format
import enum
import cv2

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [17]:
# Import related modules
import threading
import time
from Arm_Lib import Arm_Device


# Get DOFBOT object
Arm = Arm_Device()
time.sleep(.1)

# Set up gesture recognition

In [18]:
import cv2 
import time
import demjson
import pygame 
#from aip import AipBodyAnalysis
#from aip import AipSpeech
from PIL import Image, ImageDraw, ImageFont
import numpy
import ipywidgets.widgets as widgets

"""hand={'One':'number1','Two':'number2','Three':'number3','Four':'number4',
      'Five':'number5', 'Six':'number6','Seven':'number7',
      'Eight':'number8','Nine':'number9','Fist':'fist','Ok':'OK',
      'Prayer':'prayer','Congratulation':'congratulation','Honour':'honour',
      'Heart_single':'heart','Thumb_up':'thumb_up','Thumb_down':'Diss',
      'ILY':'i love you','Palm_up':'palm_up','Heart_1':'Heart_1',
      'Heart_2':'Heart_1','Heart_3':'Heart_3','Rock':'Rock','Face':'face'}

# Using your key and ID
# APPID AK SK
APP_ID = '18550528'
API_KEY = 'K6PWqtiUTKYK1fYaz13O8E3i'
SECRET_KEY = 'IDBUII1j6srF1XVNDX32I2WpuwBWczzK'


client = AipBodyAnalysis(APP_ID, API_KEY, SECRET_KEY)
"""

g_camera = cv2.VideoCapture(0)
g_camera.set(3, 640)
g_camera.set(4, 480)
g_camera.set(5, 30)  #Set frame
g_camera.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
g_camera.set(cv2.CAP_PROP_BRIGHTNESS, 40) #set brightness -64 - 64  0.0
g_camera.set(cv2.CAP_PROP_CONTRAST, 50) #set contrast -64 - 64  2.0
g_camera.set(cv2.CAP_PROP_EXPOSURE, 156) #set exposure 1.0 - 5000  156.0

ret, frame = g_camera.read()


# Define camera widget

In [19]:
image_widget = widgets.Image(format='jpeg', width=600, height=500)  # Define camera widget
display(image_widget)   
image_widget.value = bgr8_to_jpeg(frame)

Image(value=b'', format='jpeg', height='500', width='600')

# Define image widget

In [20]:
def cv2ImgAddText(img, text, left, top, textColor=(0, 255, 0), textSize=20):
    if (isinstance(img, numpy.ndarray)):  
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
   
    draw = ImageDraw.Draw(img)
    
    fontStyle = ImageFont.truetype("simhei.ttf", textSize, encoding="utf-8")
    
    draw.text((left, top), text, textColor, font=fontStyle)
    
    return cv2.cvtColor(numpy.asarray(img), cv2.COLOR_RGB2BGR)

In [21]:
def get_loc():
    prev = [None] * 6
    for i in range(len(prev)):
        ang = None
        while ang is None:
            ang = Arm.Arm_serial_servo_read(i+1)
        prev[i] = ang
    return prev

look_at = [90, 164, 18, 0, 90, 90]

"""
# Absolute positions
p_handshake_1 = [90, 95, 15, 56, 90, 70]
p_handshake_2 = [90, 95, 15, 56, 90, 145]
p_handshake_3 = [90, 100, 20, 61, 90, 145]
p_handshake_4 = [90, 95, 10, 61, 90, 145]
p_handshake_5 = [90, 95, 10, 61, 90, 80]
"""
# Relative positions
p_handshake_1 = [0, 0, 0, 0, 0, 0]  # Starting position - assuming has gotten to hand
p_handshake_2 = [0, 0, 0, 0, 0, 55]
p_handshake_3 = [0, 5, 5, 5, 0, 0]
p_handshake_4 = [0, -5, -10, 0, 0, 0]
p_handshake_3_1 = [0, 5, 10, 0, 0, 0]  # Second up shake
p_handshake_5 = [0, 0, 0, 0, 0, -45]
p_handshake_6 = [0, 0, 5, -5, 0, -10] # Return to start position

In [22]:
# Define control DOFBOT function, control No.1-No.5 servo，p=[S1,S2,S3,S4,S5]
def arm_move_6(p, s_time = 500):
    for i in range(6):
        id = i + 1
        Arm.Arm_serial_servo_write(id, p[i], s_time)
        time.sleep(.01)
    time.sleep(s_time/1000)
    
# arm_move_6 but relative (additive) to current position
def move_rel(p, prev, s_time=500):
    diff = p[:]
    for i in range(len(diff)):
        diff[i] = p[i]+prev[i]
    # print(f"Difference vector: {p}")
    # print(f"Old absolute: {prev}")
    # print(f"New absolute: {diff}")
    # Move arm to new location
    arm_move_6(diff, s_time)
    prev = get_loc()
    err = [diff[i] - prev[i] for i in len(diff)]
    # print(f"Actual absolute: {prev}")
    # print(f"Error vector: {err}")

In [ ]:
def handshake(prev, verbose=False):
    print(f"Starting position: {prev}")
    print("Moving to pos 1")
    prev = move_rel(p_handshake_1, prev, 1000) # Extend toward hand (or rather, is at hand)
    print("Moving to pos 2")
    prev = move_rel(p_handshake_2, prev, 170) # Grasp hand
    time.sleep(1)
    print("Moving to pos 3")
    prev = move_rel(p_handshake_3, prev, 150) # Shake up
    print("Moving to pos 4")
    prev = move_rel(p_handshake_4, prev, 150) # Shake down
    print("Moving to pos 3")
    prev = move_rel(p_handshake_3_1, prev, 150) # Shake up
    print("Moving to pos 4")
    prev = move_rel(p_handshake_4, prev, 150) # Shake down
    time.sleep(1/1000)
    print("Moving to pos 5")
    prev = move_rel(p_handshake_5, prev, 150) # Release hand
    time.sleep(1/1000)
    print("Moving to pos 6")
    prev = move_rel(p_handshake_3, prev, 150) # Return to pos 1 (diff vector = #6)
    print("Moving to pos home")
    Arm.Arm_serial_servo_write6_array(look_at, 1000)
    time.sleep(1)

In [23]:
global g_state_arm
g_state_arm = 0
def ctrl_arm_move(index):
    prev = get_loc()
    print(f"Updated location: {prev}")
    if index == "Handshake":
        handshake(prev)
    elif index == "Ok":
        s_time = 300
        Arm.Arm_serial_servo_write(4, 10, s_time)
        time.sleep(s_time/1000)
        Arm.Arm_serial_servo_write(4, 0, s_time)
        time.sleep(s_time/1000)
        Arm.Arm_serial_servo_write(4, 10, s_time)
        time.sleep(s_time/1000)
        Arm.Arm_serial_servo_write(4, 0, s_time)
        time.sleep(s_time/1000)
    elif index == "Seven":
        time.sleep(2)
        Arm.Arm_serial_servo_write6_array(look_at, 1000)
        time.sleep(1)
        

    global g_state_arm
    g_state_arm = 0
    

In [24]:
arm_move_6(look_at, 1000)
time.sleep(1)

In [25]:
def start_move_arm(index):
    global g_state_arm
    if g_state_arm == 0:
        closeTid = threading.Thread(target=ctrl_arm_move, args=[index])
        closeTid.setDaemon(True)
        closeTid.start()
        g_state_arm = 1

# Main process

In [ ]:
try:
    s_time = 300
    Arm.Arm_serial_servo_write(4, 10, s_time)
    time.sleep(s_time/1000)
    Arm.Arm_serial_servo_write(4, 0, s_time)
    time.sleep(s_time/1000)
    Arm.Arm_serial_servo_write(4, 10, s_time)
    time.sleep(s_time/1000)
    Arm.Arm_serial_servo_write(4, 0, s_time)
    time.sleep(s_time/1000)

    while True:
        # Get image
        ret, frame = g_camera.read()

        # Recognize gesture
        res = "Handshake"
        # img = cv2ImgAddText(frame, res, 250, 30, (0, 255, 0), 30)

        # Move arm to Absolute p_handshake_1
        # TODO: Make this move to hand once object identified
        arm_move_6([90, 95, 15, 56, 90, 70], 1500)
        time.sleep(1)
        start_move_arm("Handshake")
        #start_move_arm("Ok")
        #start_move_arm("Ok")
        #start_move_arm("Seven")

        # image_widget.value = bgr8_to_jpeg(img)
        
except KeyboardInterrupt:
    print(" Program closed! ")
    pass


nothing
nothing
nothing
nothing
 Program closed! 
nothing
 Program closed! 
nothing
nothing
nothing
nothing
 Program closed! 
nothing
nothing
nothing
nothing


In [ ]:
g_camera.release()              #Release camera object